# 네이버 영화평 감성분석
- Tokenizer
- TfidfVectorizer + LogisticRegression

In [1]:
!pip install Konlpy > /dev/null

In [3]:
import numpy as np
import pandas as pd
import warnings 
from konlpy.tag import Okt
warnings.filterwarnings("ignore")
okt = Okt()

In [5]:
from google.colab import files
up = files.upload()

Saving naver_movie_train_전처리완료.csv to naver_movie_train_전처리완료.csv


In [7]:
up.keys()

dict_keys(['naver_movie_train_전처리완료.csv'])

In [10]:
train_df = pd.read_csv("naver_movie_train_전처리완료.csv", sep = "\t")
test_df = pd.read_csv("naver_movie_test_전처리완료.csv", sep = "\t")
train_df.shape, test_df.shape

((145435, 3), (48860, 3))

In [12]:
train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


# tokenizer 함수 정의

In [13]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를',
             '으로','자','에','와','한','하다','을','ㅋㅋ','ㅠㅠ','ㅎㅎ']             

In [15]:
def okt_tokenizer(text):
    morphs = okt.morphs(text, stem = True)
    tokens = [word for word in morphs if word not in stopwords]
    return tokens

In [16]:
okt_tokenizer("열심히 일한 당신 술 마셔라!!")

['열심히', '일', '당신', '술', '마시다', '!!']

- Pipeline으로 피쳐 변환과 분류를 동시에

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [ ]:
pipeline = Pipeline([
    ("TFIDF",TfidfVectorizer(tokenizer = okt_tokenizer)),
    ("LR", LogisticRegression(random_state = 2022))
])
pipeline.fit(train_df.document, train_df.label)

In [ ]:
pipeline.score(test_df.document, test_df.label)

- 실제 데이터 테스트

In [40]:
import re
reviews = ["""이게 몰입이ㅜ안됨 스토리가 연속적이지 않고 이솝우화처럼 스토리 여러개로 분할되서 
              에피소드가 ㅈㄴ여러개임근데 서로 관련성은 전혀 없음ㄹㅇ 보다가 도중에 나온거 처음 """,
           """중간중간 코믹요소가 들어가서 너무 재밌었고 음악이, 구둣발 소리가, 제 심장을 떨리게 만들었습니다.
              정말 시간가는줄 모르고 봤구요. 엔딩이 슬프지만... 해피엔딩을 바라는건 우리의 바람일뿐이고
              어쩌면 그게 현실인것 같은.. 정말 정말 재밌게 봤습니다.. 이걸 2022년 이제야 보다니... 한번 더 보고싶네요.
              맥주 홀짝거리며 한번 더 봐야겠습니다. 도경수 배우님은 아이돌이라고는 생각도 못했는데.. 
              놀랍네요. 덕분에 도경수님 그리고 엑소 팬이 되었습니다. 흥하세요"""
]
reviews = map(lambda x :re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣]", ' ', x), reviews)

In [46]:
reviews

['이게 몰입이ㅜ안됨 스토리가 연속적이지 않고 이솝우화처럼 스토리 여러개로 분할되서 \n              에피소드가 ㅈㄴ여러개임근데 서로 관련성은 전혀 없음ㄹㅇ 보다가 도중에 나온거 처음 ',
 '중간중간 코믹요소가 들어가서 너무 재밌었고 음악이, 구둣발 소리가, 제 심장을 떨리게 만들었습니다.\n              정말 시간가는줄 모르고 봤구요. 엔딩이 슬프지만... 해피엔딩을 바라는건 우리의 바람일뿐이고\n              어쩌면 그게 현실인것 같은.. 정말 정말 재밌게 봤습니다.. 이걸 2022년 이제야 보다니... 한번 더 보고싶네요.\n              맥주 홀짝거리며 한번 더 봐야겠습니다. 도경수 배우님은 아이돌이라고는 생각도 못했는데.. \n              놀랍네요. 덕분에 도경수님 그리고 엑소 팬이 되었습니다. 흥하세요']

In [ ]:
pipeline.predict(reviews)

- 최적 파라미터 찾기
    - 매 시행시마다 한글 형태소 분석을 하느라 시간이 너무 오래 걸림
    - 최적 파라미터를 찾으려고 하면 한글 형태소 분석을 한 데이터로 할 것

In [45]:
from sklearn.model_selection import GridSearchCV
params = {
    "TFIDF__ngram_range" : [(1,1),(1,2)],
    "TFIDF__max_df" : [0.95,0.98],
    "LR__C" : [1,5,10]
}

In [ ]:
grid_pipe = GridSearchCV(pipeline, params, cv = 3, scoring = "accuracy")
%time grid_pipe.fit(train_df.document, train_df.label)

- CountVectorizer 사례에서 찾은 최적의 파라메터로 평가

In [48]:
pipeline = Pipeline([
    ("TFIDF",TfidfVectorizer(tokenizer = okt_tokenizer, max_df = 0.95, ngram_range = (1,2))),
    ("LR", LogisticRegression(random_state = 2022))
])
%time pipeline.fit(train_df.document, train_df.label)

CPU times: user 7min 14s, sys: 11.1 s, total: 7min 25s
Wall time: 7min 8s


Pipeline(steps=[('TFIDF',
                 TfidfVectorizer(max_df=0.95, ngram_range=(1, 2),
                                 tokenizer=<function okt_tokenizer at 0x7f843d325dd0>)),
                ('LR', LogisticRegression(random_state=2022))])

In [ ]:
pipeline.score(test_df.document, test_df.label)